In [1]:
import numpy as np
import random
import os
import json
import pandas as pd
import copy

In [24]:
def createData(folder_name, # directory of generated data.
               N, # number of fields
               m, # lower bound of number of field to harvest
               M, # upper bound of number of field to harvest 
               mu, # mean of d following normal distribution
               sigma, # standard deviaiton of d following normal distribution
               d_min, # min value of d to clip
               d_max, # max value of d to clip
               day_jump_random, # function returns jump day
               d_jump_random, # function returns jump d
               e_jump_random, # function returns jump e
               *args):
    
    """
    # Return
        Problem with feasible solutions
    """
    
    dictionary = dict()
    dictionary['N'] = N
    dictionary['m'] = m
    dictionary['M'] = M
    d = [0] * N
    s = [0] * N
    e = [0] * N
    
    d_samples = np.random.normal(mu, sigma, N)
    d_samples = np.clip(d_samples, d_min, d_max)
    d_samples = d_samples.tolist()
    d = [round(item) for item in d_samples]
    d_samples = copy.copy(d)
    d_indice = [*range(N)]
    
    day = day_jump_random()[0]
    days = dict()
    days[day] = list()
    d_sum = 0
    d_sums = dict()
    
    while(d_indice):
        d_index = random.sample(d_indice, k = 1)[0]
        d_indice.remove(d_index)
        if (d_sum + d[d_index] < M):
            d_sum += d[d_index]
            days[day].append(d_index)
        else:
            d_sums[day] = d_sum
            d_sum = d[d_index]
            day += day_jump_random()[0]
            days[day] = [d_index]
            
    d_sums[day] = d_sum
    if d_sum < d_min:
        d[d_index] += (d_min - d_sum)
    
    
    for index, values in days.items():
        lb_day = max(0, index - random.choices([0, 1, 2], weights=[30, 40, 30])[0])
        for value in values:
            s[value] = random.randint(lb_day, index)
            e[value] = random.randint(index, index + random.choices([0, 1, 2], weights=[50, 30, 20])[0])
            
        
    day_jump_random_info = day_jump_random()[1]
    d_jump_random_info = d_jump_random()[1]
    e_jump_random_info = e_jump_random()[1]
    arguments = locals()
    
    dictionary['d'] = d
    dictionary['s'] = s
    dictionary['e'] = e
    
    try:
        os.makedirs('./' + folder_name)
    except OSError as e:
        print("Replace existed file")
    
#     with open(folder_name + "/arguments.json", "w") as outfile:
#         json.dump(arguments, outfile)

    with open(folder_name + "/arguments.txt", "w") as arguments_file:
        arguments_file.write(str(arguments).replace(',', '\n'))
        
#     with open(folder_name + "/arguments.txt", "w") as arguments_file:
#         arguments_file.write(str(arguments))
    
    with open(folder_name + "/data.json", "w") as outfile:
        json.dump(dictionary, outfile)

In [25]:
# all random values must return the random value and its information
def day_jump_random():
    values = [1, 2, 3]; weights = [80, 15, 5]
    return random.choices(values, weights = weights)[0], {'values': values, 'weights': weights}
def d_jump_random():
    values = [0, 1, 2]; weights = [30, 40, 30]
    return random.choices(values, weights = weights)[0], {'values': values, 'weights': weights}
def e_jump_random():
    values = [0, 1, 2]; weights = [30, 40, 30]
    return random.choices(values, weights = weights)[0], {'values': values, 'weights': weights}
createData('datatest20_3', N = 20, m = 5, M = 17, mu = 11, sigma = 2, d_min = 5, d_max = 30, 
           day_jump_random = day_jump_random, 
           d_jump_random = d_jump_random, 
           e_jump_random = e_jump_random)

In [26]:
createData('datatest20_3', N = 20, m = 5, M = 17, mu = 4, sigma = 1, d_min = 2, d_max = 10, 
           day_jump_random = day_jump_random, 
           d_jump_random = d_jump_random, 
           e_jump_random = e_jump_random)

Replace existed file
